In [2]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from tqdm import tqdm

from pixyz.distributions import Normal, Bernoulli, RelaxedCategorical, Categorical

batch_size = 128
epochs = 10

if torch.cuda.is_available():plt.style.use("ggplot")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = "cuda"
else:
    device = "cpu"